# Lab 5: Textures and Materials in 3D 

---

This lab focuses on the integration of texture and textual information into 3D scenes using K3D. Students will learn how to apply 2D images (bitmaps) as textures on 3D surfaces, enabling more realistic or informative visualizations. They will also explore how to embed text into scenes for annotation or labeling — both as flat 2D overlays and as extruded 3D objects. In addition, procedural approaches will be introduced to simulate surface variation and material properties, even without image-based textures.

Through hands-on examples, students will develop an understanding of how visual appearance can be enhanced using textures and text, contributing to both the readability and realism of 3D visualizations. 

---

**Learning Objectives**

By the end of this lab, students will be able to:
- Apply 2D bitmap textures to 3D objects in K3D.
- Use text labels as visual annotations or scene elements.
- Experiment with text rendering as both flat overlays and 3D geometries.
- Understand how material appearance is affected by color, lighting, and texture mapping.

---

## Textures and Materials in 3D Graphics

Textures are 2D images (bitmaps) that are mapped onto 3D geometry to simulate surface detail, patterns, or annotations. They replace or supplement simple color assignments by adding visual richness without increasing geometric complexity. Textures can be photographic, procedurally generated, or text-based.

Material appearance refers to how objects interact with light — through color, reflectivity, texture, and transparency. In real-time 3D rendering, materials are often simplified but can include texture maps (for color, bump, or normal effects) to simulate realism.

Text rendering in 3D graphics can be done in two ways:
- 2D overlay text (text2d) which always faces the camera and behaves like a screen label.
- 3D text geometry (text) which exists as part of the 3D scene and responds to perspective and transformation.

In [1]:
!pip install k3d

---

### Example 1: Add a 2D Text Label (Overlay)

This places a fixed 2D label in the corner of the screen — great for titles or dynamic overlays.

In [3]:
import k3d

plot = k3d.plot()
label = k3d.text2d("Hello 3D World!", position=[0.2, 0.65], size=1.5, color=0x00ff00)
plot += label
plot.display()

Output()

---

### Example 2: Add a 3D Text Object into the Scene

This creates a 3D object that behaves like any mesh — you can rotate, scale, and position it in 3D space.

In [5]:
text = k3d.text("K3D TEXT", position=[0, 0, 0], scale=1.0, color=0xff0000)
plot = k3d.plot()
plot += text
plot.display()

Output()

---

### Example 3: Display a Colored Surface with Text Label

Combines a surface mesh and a 3D text object to annotate geometry in the scene.

In [25]:
import numpy as np

vertices = np.array([
    [0, 0, 0],
    [1, 0, 0],
    [1, 1, 0],
    [0, 1, 0]
], dtype=np.float32)

indices = np.array([[0, 1, 2], [0, 2, 3]], dtype=np.uint32)

plot = k3d.plot()
mesh = k3d.mesh(vertices, indices, color=0x3366ff)
plot += mesh
plot += k3d.text("Label on Surface", position=[0.3, 0.3, 0.05], scale=0.3, color=0x000000)
plot.display()

Output()

---

### Example 4: Apply Per-Vertex Color Gradient (Procedural Texture)

Demonstrates procedural "texture" using vertex color mapping instead of image-based texturing.



In [8]:
x, y = np.meshgrid(np.linspace(0, 1, 20), np.linspace(0, 1, 20))
z = np.sin(x * 6) * np.cos(y * 6)
vertices = np.stack((x.ravel(), y.ravel(), z.ravel()), axis=1)

indices = []
res = x.shape[0]
for i in range(res - 1):
    for j in range(res - 1):
        idx = i * res + j
        indices.append([idx, idx + 1, idx + res])
        indices.append([idx + 1, idx + res + 1, idx + res])
indices = np.array(indices, dtype=np.uint32)

# Color by height (Z)
z_vals = z.ravel()
norm_z = (z_vals - z_vals.min()) / z_vals.ptp()
colors = (norm_z * 0xFFFFFF).astype(np.uint32)

plot = k3d.plot()
plot += k3d.mesh(vertices=vertices.astype(np.float32), indices=indices, colors=colors)
plot.display()

NameError: name 'np' is not defined

---

### Example 5: Add Multiple 3D Text Labels to Objects

Shows how 3D text can be dynamically positioned near points or regions of interest.



In [10]:
plot = k3d.plot()

# Two spheres
plot += k3d.points([[0, 0, 0]], point_size=0.2, color=0xff0000)
plot += k3d.points([[1, 1, 0]], point_size=0.2, color=0x00ff00)

# Add text labels
plot += k3d.text("Origin", position=[0, 0, 0.2], scale=0.2)
plot += k3d.text("Point B", position=[1, 1, 0.2], scale=0.2)

plot.display()

Output()

---

###  Example 6: Create a 2D Overlay for Real-Time Measurement Display

This simulates a simple measurement or HUD-style interface in 3D scenes.



In [12]:
plot = k3d.plot()
plot += k3d.text2d("Distance: 2.4 units", position=[0.7, 0.9], size=1.0, color=0xffff00)
plot += k3d.line([[0, 0, 0], [2, 0, 0]], width=0.05, color=0xff0000)
plot.display()

Output()

---

### Example 7: Animated Labels for Moving Objects

Create a dynamic system where 3D objects move, and a text label updates its position to follow one of them in real time.

In [14]:
import k3d
import numpy as np
import time

plot = k3d.plot()

# Create a point that will move in a circular orbit
N = 100
theta = np.linspace(0, 2 * np.pi, N)
orbit = np.stack((np.cos(theta), np.sin(theta), np.zeros_like(theta)), axis=1)

# Initialize point and label
point = k3d.points(positions=np.array([[1, 0, 0]]), point_size=0.2, color=0xff0000)
label = k3d.text("Tracking...", position=[1, 0.1, 0], scale=0.15, color=0xffff00)

plot += point
plot += label
plot.display()

# Animate object + label
for i in range(N):
    pos = orbit[i]
    point.positions = np.array([pos], dtype=np.float32)
    label.position = (pos + np.array([0, 0.15, 0])).tolist()
    time.sleep(0.05)

C:\Users\cabru\anaconda3\Lib\site-packages\traittypes\traittypes.py:97: UserWarning: Given trait value dtype "int32" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Output()

---

### Example 8: Simulated Material Effects with Procedural Coloring

Mimic material surface variation (e.g. roughness or terrain elevation) using procedural vertex coloring based on curvature-like features.

In [15]:
# Create a surface using radial sine wave
x, y = np.meshgrid(np.linspace(-2, 2, 50), np.linspace(-2, 2, 50))
r = np.sqrt(x**2 + y**2)
z = 0.3 * np.sin(5 * r)

vertices = np.stack((x.ravel(), y.ravel(), z.ravel()), axis=1)

# Generate triangles
res = x.shape[0]
faces = []
for i in range(res - 1):
    for j in range(res - 1):
        idx = i * res + j
        faces.append([idx, idx + 1, idx + res])
        faces.append([idx + 1, idx + res + 1, idx + res])
faces = np.array(faces, dtype=np.uint32)

# Simulate material: color based on distance from origin
dists = np.sqrt(vertices[:, 0]**2 + vertices[:, 1]**2)
dists_norm = (dists - dists.min()) / dists.ptp()
colors = (dists_norm * 0xAAAAAA + 0x333333).astype(np.uint32)  # dark inner, light outer

plot = k3d.plot()
plot += k3d.mesh(vertices=vertices.astype(np.float32), indices=faces, colors=colors)
plot.display()

Output()

---

### Example 9: Create a 3D Info Visualization Dashboard

Create a simple data dashboard using bars (as geometry) and text labels to represent values in 3D space.

In [16]:
plot = k3d.plot()

# Simulated "bar chart" in 3D
positions = np.linspace(-2, 2, 5)
heights = np.random.uniform(0.5, 2.0, size=5)

for i, (x, h) in enumerate(zip(positions, heights)):
    # Create a vertical line as a bar
    bar = k3d.line([[x, 0, 0], [x, 0, h]], width=0.1, color=0x00ccff)
    label = k3d.text2d(f"Val {i+1}: {h:.2f}", position=[0.05, 0.9 - i * 0.07], size=1.2)
    
    plot += bar
    plot += label

plot.display()


Output()

---

# Tasks 

---

##  Task 1: Create a Text-Labeled Terrain

Combine a procedurally generated terrain (e.g. sine-based) with several 3D text labels placed at key regions (e.g. peaks or valleys).
Steps:
- Create a mesh with z = sin(x) * cos(y).
- Place text objects near the highest and lowest points.
- Use color to indicate elevation.

---

In [24]:
import numpy as np
import k3d

x = y = np.linspace(-3, 3, 50)
X, Y = np.meshgrid(x, y)
Z = np.sin(X) * np.cos(Y)


vertices = np.column_stack((X.ravel(), Y.ravel(), Z.ravel())).astype(np.float32)

n = X.shape[0]
indices = []
for i in range(n - 1):
    for j in range(n - 1):
        a = i * n + j
        b = a + 1
        c = a + n
        d = c + 1
        indices += [[a, c, b], [b, c, d]]
indices = np.array(indices, dtype=np.uint32)


Z_flat = Z.ravel()
z_norm = (Z_flat - Z_flat.min()) / (Z_flat.max() - Z_flat.min())


terrain = k3d.mesh(vertices, indices, attribute=z_norm, color_map=k3d.basic_color_maps.Jet)


peak_index = np.argmax(Z_flat)
valley_index = np.argmin(Z_flat)
peak_pos = vertices[peak_index]
valley_pos = vertices[valley_index]

peak_label = k3d.text('Peak', position=(peak_pos[0], peak_pos[1], peak_pos[2] + 0.2), color=0xff0000, size=0.6)
valley_label = k3d.text('Valley', position=(valley_pos[0], valley_pos[1], valley_pos[2] + 0.2), color=0x0000ff, size=0.6)


plot = k3d.plot()
plot += terrain
plot += peak_label
plot += valley_label
plot.display()


Output()

## Task 2: Simulate a Control Panel Using 2D Overlays

Use text2d elements to display interface elements such as coordinates, frame rate, or simulation info.
Steps:
- Create a 3D object (e.g. rotating cube or orbiting point).
- Add 2D overlay text that updates or labels information.

---

In [52]:
import numpy as np
import k3d
import time


vertices = 0.5 * np.array([
    [-1, -1, -1],
    [ 1, -1, -1],
    [ 1,  1, -1],
    [-1,  1, -1],
    [-1, -1,  1],
    [ 1, -1,  1],
    [ 1,  1,  1],
    [-1,  1,  1]
], dtype=np.float32)

faces = np.array([
    [0, 1, 2], [0, 2, 3],
    [4, 5, 6], [4, 6, 7],
    [0, 1, 5], [0, 5, 4],
    [2, 3, 7], [2, 7, 6],
    [1, 2, 6], [1, 6, 5],
    [0, 3, 7], [0, 7, 4]
], dtype=np.uint32)

# === Setup K3D plot ===
plot = k3d.plot()
cube = k3d.mesh(vertices, faces, color=0x3366ff, wireframe=False)  
plot += cube

angle_label = k3d.text2d("Angle: 0°", position=[0.05, 0.9], size=1.2, color=0x00ff00)  
fps_label = k3d.text2d("FPS: 0", position=[0.05, 0.85], size=1.0, color=0xffffff)
plot += angle_label
plot += fps_label
plot.display()


frame_times = []
last_time = time.time()

for frame in range(180):
    angle = np.deg2rad(frame * 2)
    rotation_matrix = np.array([
        [ np.cos(angle), 0, np.sin(angle)],
        [ 0,             1, 0           ],
        [-np.sin(angle), 0, np.cos(angle)]
    ])
    rotated = vertices @ rotation_matrix.T
    cube.vertices = rotated.astype(np.float32)


    angle_label.text = f"Angle: {int(np.rad2deg(angle)) % 360}°"

    now = time.time()
    frame_times.append(now - last_time)
    last_time = now
    if len(frame_times) > 30:
        frame_times.pop(0)
    avg_dt = np.mean(frame_times)
    fps = 1.0 / avg_dt if avg_dt > 0 else 0.0
    fps_label.text = f"FPS: {fps:.1f}"

    time.sleep(0.03)


Output()

### Task 3: Procedural Color Texture on a Curved Surface

Visualize a mathematical surface (e.g. paraboloid or sine wave) with a color gradient based on height.
Steps:
- Generate mesh and assign per-vertex color.
- Use normalization + color encoding to simulate texture.
- Compare with flat color.

---

In [54]:
import numpy as np
import k3d


res = 50
x = np.linspace(-2, 2, res)
y = np.linspace(-2, 2, res)
X, Y = np.meshgrid(x, y)
Z = np.sin(X**2 + Y**2) 


vertices = np.stack([X, Y, Z], axis=-1).reshape(-1, 3)

indices = []
for i in range(res - 1):
    for j in range(res - 1):
        a = i * res + j
        b = a + 1
        c = a + res
        d = c + 1
        indices += [[a, b, c], [b, d, c]]
indices = np.array(indices, dtype=np.uint32)


z_vals = vertices[:, 2]
z_norm = (z_vals - z_vals.min()) / (z_vals.max() - z_vals.min())
colors = (255 * z_norm).astype(np.uint8)
color_rgb = np.stack([colors, 255 - colors, 128 + 0*colors], axis=1)
color_uint = (color_rgb[:, 0] << 16) + (color_rgb[:, 1] << 8) + color_rgb[:, 2]


mesh = k3d.mesh(vertices, indices, colors=color_uint.astype(np.uint32))
plot = k3d.plot()
plot += mesh
plot.display()


Output()

## Task 4: Label a 3D Point Cloud or Model

Load or create a point cloud and annotate selected points with 3D text.
Steps:
- Generate or import a 3D point set.
- Add 3D text near selected or randomly chosen points.
- Group related points with the same label color.

---

In [62]:
import numpy as np
import k3d
from sklearn.cluster import KMeans


N = 200
points = np.random.uniform(-1, 1, (N, 3)).astype(np.float32)


kmeans = KMeans(n_clusters=3, n_init=5)
labels = kmeans.fit_predict(points)

group_colors = [0xff0000, 0x00ff00, 0x0000ff]  # red, green, blue
colors = np.array([group_colors[l] for l in labels], dtype=np.uint32)


cloud = k3d.points(positions=points, colors=colors, point_size=0.1)


texts = []
for i in range(3):
    center = kmeans.cluster_centers_[i]
    label = k3d.text(f'Group {i+1}', position=(center + 0.05).tolist(), color=group_colors[i], size=0.4)
    texts.append(label)


plot = k3d.plot()
plot += cloud
for label in texts:
    plot += label
plot.display()


C:\Users\cabru\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


Output()

## Task 5: Create a 3D Labeled Coordinate System Viewer

Build a labeled 3D coordinate reference frame with axis lines and corresponding text labels for X, Y, and Z. Add arrows or points to enhance orientation.
Steps:
- Use k3d.line() to create three colored axes: X (red), Y (green), Z (blue)
- Add k3d.text() labels near the end of each axis: "X", "Y", and "Z".
- Place small k3d.points() or arrows on the axis ends.
- Style everything for clarity: consistent colors, readable scales, centered origin.

The outcome should be a reusable 3D coordinate reference tool that can be included in any future visualizations or used as an orientation helper. 

---

In [80]:
import numpy as np
import k3d


L = 1.0


x_axis = k3d.line(np.array([[0, 0, 0], [L, 0, 0]]), color=0xff0000, width=0.01)  # Red X
y_axis = k3d.line(np.array([[0, 0, 0], [0, L, 0]]), color=0x00ff00, width=0.01)  # Green Y
z_axis = k3d.line(np.array([[0, 0, 0], [0, 0, L]]), color=0x0000ff, width=0.01)  # Blue Z


x_label = k3d.text('X', position=[L + 0.05, 0, 0], color=0xff0000, size=0.5)
y_label = k3d.text('Y', position=[0, L + 0.05, 0], color=0x00ff00, size=0.5)
z_label = k3d.text('Z', position=[0, 0, L + 0.05], color=0x0000ff, size=0.5)


tip_points = k3d.points(positions=np.array([
    [L, 0, 0],
    [0, L, 0],
    [0, 0, L]
]), colors=np.array([0xff0000, 0x00ff00, 0x0000ff], dtype=np.uint32), point_size=0.12)


plot = k3d.plot()
plot += x_axis
plot += y_axis
plot += z_axis
plot += tip_points
plot += x_label
plot += y_label
plot += z_label
plot.display()


Output()

### Task 6: Build a Scene Legend Using Floating 2D Labels

Create a floating legend using multiple text2d elements in the corner of the screen, describing elements in your 3D scene (e.g., red = error, green = safe, blue = neutral).
Steps:
- Create 3D objects with different colors (e.g., spheres, points, bars).
- Place text2d() labels in the top-right of the screen to describe what each color means.
- Add a mini title (Scene Legend) in bold or larger size.
- Use consistent formatting and positions (e.g., [0.7, 0.9], [0.7, 0.85], etc.)

The outcome shpuld be an interactive 3D visualization with an integrated floating legend to help users interpret the scene intuitively — useful for dashboards, reports, or presentation visuals.

---

In [78]:
import numpy as np
import k3d

plot = k3d.plot()


objects = []
colors = [0xff0000, 0x00ff00, 0x0000ff]
positions = [[-0.5, 0, 0], [0, 0, 0], [0.5, 0, 0]]

for color, pos in zip(colors, positions):
    sphere = k3d.points(positions=[pos], point_size=0.3, color=color, shader='3d')
    plot += sphere


legend_title = k3d.text2d('Scene Legend', position=[0.75, 0.95], color=0x000000, size=1.5)
legend_red   = k3d.text2d('🔴  Red = Error',  position=[0.75, 0.85], color=0xff0000, size=1.2)
legend_green = k3d.text2d('🟢  Green = Safe',  position=[0.75, 0.75], color=0x00ff00, size=1.2)
legend_blue  = k3d.text2d('🔵  Blue = Neutral', position=[0.75, 0.65], color=0x0000ff, size=1.2)


plot += legend_title
plot += legend_red
plot += legend_green
plot += legend_blue

plot.display()


Output()